## Checking network updates for different batches ##

### shuffling the dataset at each optimization step ###

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm_notebook as tqdm
tf.keras.backend.set_floatx('float64')
from misc import *
from collections import deque
from datetime import datetime
import random

In [2]:
def Prob(alpha, beta, n):
    p0 = np.exp(-(alpha-beta)**2)
    if n == 0:
        return p0
    else:
        return 1-p0

def qval(beta, n, guess):
    #dolinar guessing rule (= max-likelihood for L=1, careful sign of \beta)
    alpha = 0.4
    pn = np.sum([Prob(g*alpha, beta, n) for g in [-1,1]])
    return Prob(guess*alpha, beta, n)/pn


** so it apperars to stuck at the mean value of the sum of all rewards, which is not what we want **

In [4]:

class Net(tf.keras.Model):
    def __init__(self, valreg=0.01, seed_val=0.1, dropout=0.01):
        super(Net,self).__init__()
        self.dropout = dropout
        self.l1 = Dense(10, input_shape=(3,),kernel_initializer=tf.random_uniform_initializer(minval=-seed_val, maxval=seed_val),
        bias_initializer = tf.random_uniform_initializer(minval=-seed_val, maxval=seed_val),
        kernel_regularizer=tf.keras.regularizers.l1(valreg),
    activity_regularizer=tf.keras.regularizers.l2(valreg))

        self.l2 = Dense(330, kernel_regularizer=tf.keras.regularizers.l1(valreg),
    activity_regularizer=tf.keras.regularizers.l2(valreg),
    kernel_initializer=tf.random_uniform_initializer(minval=-seed_val, maxval=seed_val),
    bias_initializer = tf.random_uniform_initializer(minval=-seed_val, maxval=seed_val))
        self.l3 = Dense(330, kernel_regularizer=tf.keras.regularizers.l1(valreg),
    activity_regularizer=tf.keras.regularizers.l2(valreg),
    kernel_initializer=tf.random_uniform_initializer(minval=-seed_val, maxval=seed_val),
    bias_initializer = tf.random_uniform_initializer(minval=-seed_val, maxval=seed_val))

        self.l4 = Dense(33, kernel_regularizer=tf.keras.regularizers.l1(valreg),
    activity_regularizer=tf.keras.regularizers.l2(valreg),
    kernel_initializer=tf.random_uniform_initializer(minval=-seed_val, maxval=seed_val),
    bias_initializer = tf.random_uniform_initializer(minval=-seed_val, maxval=seed_val))

        self.l5 = Dense(1, kernel_regularizer=tf.keras.regularizers.l1(valreg),
    activity_regularizer=tf.keras.regularizers.l2(valreg),
    kernel_initializer=tf.random_uniform_initializer(minval=-seed_val, maxval=seed_val),
    bias_initializer = tf.random_uniform_initializer(minval=-seed_val, maxval=seed_val))

    def call(self, input):
        feat = tf.nn.relu(self.l1(input))
        #feat = tf.nn.dropout(feat, rate=self.dropout)
        #feat = tf.nn.relu(self.l4(feat))
        feat = tf.nn.sigmoid(self.l5(feat))
        return feat

    def predict(self, history_one):
        inp = np.expand_dims(history_one, axis=0)
        return np.squeeze(self(inp).numpy())

    def __str__(self):
        return self.name


In [5]:
class DataSet():
    def __init__(self, rpbgo=10, nbetas=10):
        #rpbgo = rewards per beta-guess-outcome
        self.nbetas=nbetas
        self.rpbgo=rpbgo
        self.betas = np.linspace(-1.5,1.5,nbetas)
        self.size = self.nbetas*self.rpbgo*4.
        d = []
        for b in self.betas:
            for outcome in [0.,1,]:
                for guess in [-1.,1.]:
                    for k in range(self.rpbgo):
                        mean_bernoulli = qval(b, outcome, guess)
                        reward = np.random.choice([1.,0,],1,p=[mean_bernoulli, 1.-mean_bernoulli])[0]

                        d.append([b, outcome, guess, reward])
        self.data_unsplitted = np.array(d)

    def batched_shuffled_dataset(self,splits):
        datacopy = self.data_unsplitted.copy()
        np.random.shuffle(datacopy)
        datacopy = np.split(datacopy, splits + len(datacopy)%splits)
        return datacopy

In [8]:
opt = tf.keras.optimizers.Adam(lr=0.01)
net = Net()
dataset = DataSet(nbetas=2)
data_training = dataset.batched_shuffled_dataset(dataset.size/1.)


In [9]:
data_training[0]

array([[-1.5,  0. ,  1. ,  0. ]])

In [11]:
opt = tf.keras.optimizers.Adam(lr=0.01)
net = Net()
dataset = DataSet(nbetas=10)
net(data_training[0]) #calling the net


<tf.Tensor: shape=(1, 1), dtype=float64, numpy=array([[0.50919727]])>

In [12]:

val_vars = {"0":[]}
for k in net.trainable_variables:
    val_vars["0"].append(k)
    

In [16]:
data_training = dataset.batched_shuffled_dataset(dataset.size/1.)


In [25]:
net= Net()
preds = net(mini_batch[:,[0,1,2]]) #make predictions in batch
optimizer = tf.keras.optimizers.Adam(lr=0.01)

In [57]:
for mini_batch in data_training:

    with tf.GradientTape() as tape:
        tape.watch(net.trainable_variables)
        preds = net(mini_batch[:,[0,1,2]]) #make predictions in batch
        rews = mini_batch[:,3]
        loss = tf.keras.losses.MSE(rews,preds)
        loss = tf.reduce_mean(loss)
        grads = tape.gradient(loss, net.trainable_variables)
        optimizer.apply_gradients(zip(grads, net.trainable_variables))


In [54]:
net(mini_batch[:,[0,1,2]]) #make predictions in batch


<tf.Tensor: shape=(1, 1), dtype=float64, numpy=array([[0.56669078]])>

In [55]:
net(np.array([[0.4,3.4,2.]]))

<tf.Tensor: shape=(1, 1), dtype=float64, numpy=array([[0.56660826]])>

In [58]:
for k in net.trainable_variables:
    print(k)

<tf.Variable 'net_5/dense_25/kernel:0' shape=(3, 10) dtype=float64, numpy=
array([[-0.58073675,  0.03216899, -0.4880766 , -0.37384412, -0.11139388,
        -0.41602594, -0.53963487, -0.01088861, -0.56441345,  0.68737243],
       [-1.39886675,  0.04447953, -1.15909707, -0.74666179, -0.53675349,
        -0.9244858 , -1.40088216, -0.08039279, -0.82957052, -1.58448969],
       [ 0.55814614,  0.01458732,  0.49749183,  0.25534114, -0.0334608 ,
         0.40996346, -0.58311595, -0.01271856, -0.48118621, -0.70650789]])>
<tf.Variable 'net_5/dense_25/bias:0' shape=(10,) dtype=float64, numpy=
array([-0.30910146, -0.17539182, -0.27548548, -0.29169663, -0.2244193 ,
       -0.24866701, -0.21947244, -0.09898953, -0.09402386, -0.36547767])>
<tf.Variable 'net_5/dense_29/kernel:0' shape=(10, 1) dtype=float64, numpy=
array([[-0.97316333],
       [-0.01879339],
       [-0.56305358],
       [-0.30110529],
       [ 0.14277623],
       [-0.47553278],
       [ 0.8330737 ],
       [ 0.05496462],
       [ 0.483

In [50]:

for epoch in tqdm(range(1,4)):
        with tf.GradientTape() as tape:
            tape.watch(net.trainable_variables)
            preds = net(mini_batch[:,[0,1,2]])
            labs = mini_batch[:,3]
            loss = tf.keras.losses.MSE(labs,preds)
            loss = tf.reduce_mean(loss)
         #   loss = tf.reduce_sum(tf.square(preds-labs))/len(preds)
            grads = tape.gradient(loss,net.trainable_variables)
            opt.apply_gradients(zip(grads,net.trainable_variables))
            
            for k,j in zip(net.trainable_variables, val_vars["0"]):
                print(k-j)
    new=[]
    for k in net.trainable_variables:
        new.append(k)
    val_vars[str(epoch)]=new
    del new
    

tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(3, 10), dtype=float64)
tf.Tensor([0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(10,), dtype=float64)
tf.Tensor(
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]], shape=(10, 1), dtype=float64)
tf.Tensor([0.], shape=(1,), dtype=float64)
tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(3, 10), dtype=float64)
tf.Tensor([0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(10,), dtype=float64)
tf.Tensor(
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]], shape=(10, 1), dtype=float64)
tf.Tensor([0.], shape=(1,), dtype=float64)
tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(3, 10), dtype=float64)
tf.Tensor([0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(10,), dtype=float64)
tf.Tensor(
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.

tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(3, 10), dtype=float64)
tf.Tensor([0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(10,), dtype=float64)
tf.Tensor(
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]], shape=(10, 1), dtype=float64)
tf.Tensor([0.], shape=(1,), dtype=float64)
tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(3, 10), dtype=float64)
tf.Tensor([0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(10,), dtype=float64)
tf.Tensor(
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]], shape=(10, 1), dtype=float64)
tf.Tensor([0.], shape=(1,), dtype=float64)
tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(3, 10), dtype=float64)
tf.Tensor([0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(10,), dtype=float64)
tf.Tensor(
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.

tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(3, 10), dtype=float64)
tf.Tensor([0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(10,), dtype=float64)
tf.Tensor(
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]], shape=(10, 1), dtype=float64)
tf.Tensor([0.], shape=(1,), dtype=float64)
tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(3, 10), dtype=float64)
tf.Tensor([0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(10,), dtype=float64)
tf.Tensor(
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]], shape=(10, 1), dtype=float64)
tf.Tensor([0.], shape=(1,), dtype=float64)
tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(3, 10), dtype=float64)
tf.Tensor([0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(10,), dtype=float64)
tf.Tensor(
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.

tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(3, 10), dtype=float64)
tf.Tensor([0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(10,), dtype=float64)
tf.Tensor(
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]], shape=(10, 1), dtype=float64)
tf.Tensor([0.], shape=(1,), dtype=float64)
tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(3, 10), dtype=float64)
tf.Tensor([0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(10,), dtype=float64)
tf.Tensor(
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]], shape=(10, 1), dtype=float64)
tf.Tensor([0.], shape=(1,), dtype=float64)
tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(3, 10), dtype=float64)
tf.Tensor([0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(10,), dtype=float64)
tf.Tensor(
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.

tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(3, 10), dtype=float64)
tf.Tensor([0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(10,), dtype=float64)
tf.Tensor(
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]], shape=(10, 1), dtype=float64)
tf.Tensor([0.], shape=(1,), dtype=float64)
tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(3, 10), dtype=float64)
tf.Tensor([0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(10,), dtype=float64)
tf.Tensor(
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]], shape=(10, 1), dtype=float64)
tf.Tensor([0.], shape=(1,), dtype=float64)
tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(3, 10), dtype=float64)
tf.Tensor([0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(10,), dtype=float64)
tf.Tensor(
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.

...

tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(3, 10), dtype=float64)
tf.Tensor([0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(10,), dtype=float64)
tf.Tensor(
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]], shape=(10, 1), dtype=float64)
tf.Tensor([0.], shape=(1,), dtype=float64)
tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(3, 10), dtype=float64)
tf.Tensor([0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(10,), dtype=float64)
tf.Tensor(
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]], shape=(10, 1), dtype=float64)
tf.Tensor([0.], shape=(1,), dtype=float64)
tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(3, 10), dtype=float64)
tf.Tensor([0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(10,), dtype=float64)
tf.Tensor(
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.

tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(3, 10), dtype=float64)
tf.Tensor([0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(10,), dtype=float64)
tf.Tensor(
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]], shape=(10, 1), dtype=float64)
tf.Tensor([0.], shape=(1,), dtype=float64)
tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(3, 10), dtype=float64)
tf.Tensor([0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(10,), dtype=float64)
tf.Tensor(
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]], shape=(10, 1), dtype=float64)
tf.Tensor([0.], shape=(1,), dtype=float64)
tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(3, 10), dtype=float64)
tf.Tensor([0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(10,), dtype=float64)
tf.Tensor(
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.

tf.Tensor(
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]], shape=(10, 1), dtype=float64)
tf.Tensor([0.], shape=(1,), dtype=float64)
tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(3, 10), dtype=float64)
tf.Tensor([0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(10,), dtype=float64)
tf.Tensor(
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]], shape=(10, 1), dtype=float64)
tf.Tensor([0.], shape=(1,), dtype=float64)
tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(3, 10), dtype=float64)
tf.Tensor([0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(10,), dtype=float64)
tf.Tensor(
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]], shape=(10, 1), dtype=float64)
tf.Tensor([0.], shape=(1,), dtype=float64)
tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(3, 10), dtype=flo

tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(3, 10), dtype=float64)
tf.Tensor([0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(10,), dtype=float64)
tf.Tensor(
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]], shape=(10, 1), dtype=float64)
tf.Tensor([0.], shape=(1,), dtype=float64)
tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(3, 10), dtype=float64)
tf.Tensor([0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(10,), dtype=float64)
tf.Tensor(
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]], shape=(10, 1), dtype=float64)
tf.Tensor([0.], shape=(1,), dtype=float64)
tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(3, 10), dtype=float64)
tf.Tensor([0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(10,), dtype=float64)
tf.Tensor(
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.

tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(3, 10), dtype=float64)
tf.Tensor([0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(10,), dtype=float64)
tf.Tensor(
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]], shape=(10, 1), dtype=float64)
tf.Tensor([0.], shape=(1,), dtype=float64)
tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(3, 10), dtype=float64)
tf.Tensor([0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(10,), dtype=float64)
tf.Tensor(
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]], shape=(10, 1), dtype=float64)
tf.Tensor([0.], shape=(1,), dtype=float64)
tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(3, 10), dtype=float64)
tf.Tensor([0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(10,), dtype=float64)
tf.Tensor(
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.

In [44]:
val_vars["0"]

[<tf.Variable 'net_6/dense_30/kernel:0' shape=(3, 10) dtype=float64, numpy=
 array([[ 0.41233966,  0.05918468,  0.01800167, -0.67080881,  0.126151  ,
          0.03179857,  0.25682554,  0.05818007,  0.07506869,  0.02518714],
        [ 0.87901001, -0.02837624, -0.11797021, -1.10972795, -0.20730267,
         -0.05666528,  0.34716108, -0.05732729, -0.17670183, -0.05827561],
        [ 0.50792106, -0.00755032, -0.01045144, -0.8373137 , -0.20297293,
         -0.02108069, -0.28816896,  0.11271276,  0.06625836, -0.02420185]])>,
 <tf.Variable 'net_6/dense_30/bias:0' shape=(10,) dtype=float64, numpy=
 array([ 0.18381482, -0.13482938, -0.11077148, -0.13447737, -0.21763224,
        -0.13853298, -0.01595433, -0.20147533, -0.21222043, -0.09131815])>,
 <tf.Variable 'net_6/dense_34/kernel:0' shape=(10, 1) dtype=float64, numpy=
 array([[ 5.59271851e-01],
        [ 2.68922446e-02],
        [ 3.72139036e-02],
        [ 1.43878075e+00],
        [-7.04661582e-02],
        [ 3.75009472e-05],
        [-1.315

In [45]:
val_vars["1"]

[<tf.Variable 'net_6/dense_30/kernel:0' shape=(3, 10) dtype=float64, numpy=
 array([[ 0.41233966,  0.05918468,  0.01800167, -0.67080881,  0.126151  ,
          0.03179857,  0.25682554,  0.05818007,  0.07506869,  0.02518714],
        [ 0.87901001, -0.02837624, -0.11797021, -1.10972795, -0.20730267,
         -0.05666528,  0.34716108, -0.05732729, -0.17670183, -0.05827561],
        [ 0.50792106, -0.00755032, -0.01045144, -0.8373137 , -0.20297293,
         -0.02108069, -0.28816896,  0.11271276,  0.06625836, -0.02420185]])>,
 <tf.Variable 'net_6/dense_30/bias:0' shape=(10,) dtype=float64, numpy=
 array([ 0.18381482, -0.13482938, -0.11077148, -0.13447737, -0.21763224,
        -0.13853298, -0.01595433, -0.20147533, -0.21222043, -0.09131815])>,
 <tf.Variable 'net_6/dense_34/kernel:0' shape=(10, 1) dtype=float64, numpy=
 array([[ 5.59271851e-01],
        [ 2.68922446e-02],
        [ 3.72139036e-02],
        [ 1.43878075e+00],
        [-7.04661582e-02],
        [ 3.75009472e-05],
        [-1.315

In [12]:
val_vars["1"]

[<tf.Variable 'net/dense/kernel:0' shape=(3, 10) dtype=float64, numpy=
 array([[-0.34671915,  0.01565517, -0.56101933,  0.39043954,  0.03643981,
          0.03763004,  0.0473113 , -0.02216232,  0.03839598, -0.02899709],
        [ 0.54158266, -0.1434082 , -1.57416758,  0.58483432,  0.04409883,
         -0.05302212,  0.02718095, -0.00381758, -0.03439437, -0.05763174],
        [ 0.05591131,  0.01218305, -0.60543944,  0.58812936,  0.00190351,
         -0.0317982 , -0.08290071,  0.06130308, -0.02023079, -0.01312849]])>,
 <tf.Variable 'net/dense/bias:0' shape=(10,) dtype=float64, numpy=
 array([ 0.04931027, -0.16737943, -0.49870793,  0.22108472, -0.13403139,
        -0.12089975, -0.15557635, -0.17490481, -0.09143738, -0.14524918])>,
 <tf.Variable 'net/dense_4/kernel:0' shape=(10, 1) dtype=float64, numpy=
 array([[-0.20561308],
        [ 0.00963932],
        [ 0.97706653],
        [ 0.47374353],
        [-0.0381405 ],
        [-0.00739492],
        [ 0.00802625],
        [ 0.05886739],
      